# Update PacBio tables

This notebook updates the 'sample' table of available PacBio flowcells, 'sample_set' table (one or more flowcells per participant), and 'participant' table (list of unique individuals discovered).

To auto-populate these tables, this notebook scans files in the gs://broad-gp-pacbio bucket and extracts relevant metadata from the \*.subreadset.xml files.

If changes were made to the 'sample' table in Terra, we take care not to overwrite those changes. If one wishes to restore those entries to their original values, the rows should first be deleted from the Terra table.

All other tables are auto-generated based on the 'sample' table.

## Setup

Install and import some packages that we're going to need.  Set up some helper functions.

In [1]:
#%pip install --use-feature=2020-resolver --upgrade pip pandas_gbq google-cloud-storage google-cloud-bigquery fastnumbers xmltodict > /dev/null 2>/dev/null

In [2]:
import os
import re
import math
import hashlib

import pandas as pd
import firecloud.api as fapi

from google.cloud import bigquery
from google.cloud import storage
from google.api_core.exceptions import NotFound

from collections import OrderedDict

import xmltodict
import pprint

In [3]:
def traverse_xml(key, xml):
    tables = []
    table = {}

    for k in xml:
        if 'xmlns' in k or 'xsi' in k:
            continue

        v = xml[k]

        k = re.sub('^@|^#|^pbds:|^pbbase:|^pbmeta:|^pbsample:', '', k)

        l = []
        if isinstance(v, str) or isinstance(v, dict):
            l = [v]
        elif isinstance(v, list):
            l = v

        for va in l:
            if isinstance(va, str):
                table[k] = v
            if isinstance(va, dict):
                f = traverse_xml(k, va)
                tables.extend(f)

    if len(table) > 0:
        tables.append({key: table})

    return tables


def combine(tables):
    combined_tables = {}

    for table in tables:
        for k in table:
            if k not in combined_tables:
                combined_tables[k] = []

            combined_tables[k].append(table[k])

    return combined_tables


def load_xmls(gcs_buckets):
    storage_client = storage.Client()
    schemas = OrderedDict()

    ts = []
    for gcs_bucket in gcs_buckets:
        blobs = storage_client.list_blobs(re.sub("^gs://", "", gcs_bucket))

        for blob in blobs:
            if 'subreadset.xml' in blob.name:
                xml = blob.download_as_string()
                doc = xmltodict.parse(xml)

                t = combine(traverse_xml('root', doc))
                t['Files'] = {
                    'input_dir': os.path.dirname(gcs_bucket + "/" + blob.name),
                    
                    'subreadset.xml': gcs_bucket + "/" + blob.name,
                    'subreads.bam': gcs_bucket + "/" + re.sub("et.xml", ".bam", blob.name),
                    'subreads.bam.pbi': gcs_bucket + "/" + re.sub("et.xml", ".bam.pbi", blob.name),
                    
                    'consensusreadset.xml': "",
                    'ccs_reports.txt': "",
                    'reads.bam': "",
                    'reads.bam.pbi': ""
                }
                ts.append(t)
            elif 'consensusreadset.xml' in blob.name:
                xml = blob.download_as_string()
                doc = xmltodict.parse(xml)

                t = combine(traverse_xml('root', doc))
                t['Files'] = {
                    'input_dir': os.path.dirname(gcs_bucket + "/" + blob.name),
                    
                    'subreadset.xml': "",
                    'subreads.bam': "",
                    'subreads.bam.pbi': "",

                    'consensusreadset.xml': gcs_bucket + "/" + blob.name,
                    'ccs_reports.txt': gcs_bucket + "/" + re.sub(".consensusreadset.xml", ".ccs_reports.txt", blob.name),
                    'reads.bam': gcs_bucket + "/" + re.sub(".consensusreadset.xml", ".reads.bam", blob.name),
                    'reads.bam.pbi': gcs_bucket + "/" + re.sub(".consensusreadset.xml", ".reads.bam.pbi", blob.name)
                }
                ts.append(t)

    return ts


def upload_data(namespace, workspace, tbl):
    # delete old sample set
    ss_old = fapi.get_entities(namespace, workspace, f'sample_set').json()
    sample_sets = list(map(lambda e: e['name'], ss_old))
    f = [fapi.delete_sample_set(namespace, workspace, sample_set_index) for sample_set_index in sample_sets]
    
    # delete old samples
    s_old = fapi.get_entities(namespace, workspace, 'sample').json()
    samples = list(map(lambda e: e['name'], s_old))
    f = [fapi.delete_sample(namespace, workspace, sample_index) for sample_index in samples]

    # upload new samples
    a = fapi.upload_entities(namespace, workspace, entity_data=tbl.to_csv(index=False, sep="\t"), model='flexible')

    if a.status_code == 200:
        print(f'Uploaded {len(tbl)} rows successfully.')
    else:
        print(a.json())

    # upload new sample set
    ss = tbl.filter(['bio_sample'], axis=1).drop_duplicates()
    ss.columns = [f'entity:sample_set_id']
    
    b = fapi.upload_entities(namespace, workspace, entity_data=ss.to_csv(index=False, sep="\t"), model='flexible')
    if b.status_code == 200:
        print(f'Uploaded {len(ss)} sample sets successfully.')
    else:
        print(b.json())
    
    # upload membership set
    ms = tbl.filter(['bio_sample', 'entity:sample_id'], axis=1).drop_duplicates()
    ms.columns = [f'membership:sample_set_id', f'sample']
    
    c = fapi.upload_entities(namespace, workspace, entity_data=ms.to_csv(index=False, sep="\t"), model='flexible')
    if c.status_code == 200:
        print(f'Uploaded {len(ms)} sample set members successfully.')
    else:
        print(c.json())


def load_ccs_report(ccs_report_path):
    d = {
        'ZMWs input': "",
        'ZMWs pass filters': "",
        'ZMWs fail filters': "",
        'ZMWs shortcut filters': "",
        'ZMWs with tandem repeats': "",
        'Below SNR threshold': "",
        'Median length filter': "",
        'Lacking full passes': "",
        'Heteroduplex insertions': "",
        'Coverage drops': "",
        'Insufficient draft cov': "",
        'Draft too different': "",
        'Draft generation error': "",
        'Draft above --max-length': "",
        'Draft below --min-length': "",
        'Reads failed polishing': "",
        'Empty coverage windows': "",
        'CCS did not converge': "",
        'CCS below minimum RQ': "",
        'Unknown error': ""
    }
    
    if ccs_report_path != "":
        storage_client = storage.Client()

        ccs_report = re.sub("^gs://", "", e['Files']['ccs_reports.txt']).split("/")
        blobs = storage_client.list_blobs(ccs_report[0], prefix="/".join(ccs_report[1:]))

        for blob in blobs:
            blob.download_to_filename("ccs_report.txt")

            file = open("ccs_report.txt", "r")

            d = {}
            for line in file:
                if len(line) > 1 and 'Exclusive counts for ZMWs' not in line:
                    a = line.rstrip().split(":")

                    k = a[0].rstrip()
                    v = float(re.sub(" ", "", re.sub(" \(.*$", "", a[1])))

                    if k not in d:
                        d[k] = 0.0;

                    d[k] = d[k] + v

            break
            
    return d

## Environment

Set up our environment (Terra namespace, workspace, and the location of PacBio bucket(s)).

In [4]:
namespace = os.environ['GOOGLE_PROJECT']
workspace = os.environ['WORKSPACE_NAME']
default_bucket = os.environ['WORKSPACE_BUCKET']

gcs_buckets_pb = ['gs://broad-gp-pacbio']

In [5]:
print(namespace)
print(workspace)
print(default_bucket)

broad-firecloud-dsde-methods
dsp-pacbio
gs://fc-059b0ca3-b033-4b36-a880-4b5ceb36af3b


## Retrieve existing sample table from Terra

If it exists, retrieve the 'sample' table from this workspace.

In [6]:
ent_old = fapi.get_entities(namespace, workspace, 'sample').json()

if len(ent_old) > 0:
    tbl_old = pd.DataFrame(list(map(lambda e: e['attributes'], ent_old)))
    tbl_old["entity:sample_id"] = list(map(lambda f: f['name'], ent_old))

In [7]:
tbl_old

,application,is_ccs,total_length,bio_sample,well_name,description,gcs_input_dir,well_sample,num_records,subreads_bam,...,is_corrected,instrument,subreads_pbi,zmws_input,ccs_pbi,zmws_fail,zmws_shortcut_filters,zmws_pass,ccs_bam,entity:sample_id
0,unknown,true,138425825314,SM-KM1PP,D01,unknown,gs://broad-gp-pacbio/r64020_20201212_014737/4_D01,SM-KM1PP,11439743,gs://broad-gp-pacbio/r64020_20201212_014737/4_...,...,false,Sequel II 64020,gs://broad-gp-pacbio/r64020_20201212_014737/4_...,NaN,NaN,NaN,NaN,NaN,NaN,01300ed1-77fe-41bb-92c2-7907dfb910c5
1,unknown,unknown,97407535687,VerilySK-BR-3CCS,A01,unknown,gs://broad-gp-pacbio/r64020_20190425_132745/1_A01,VerilySK-BR-3CCS,9703409,gs://broad-gp-pacbio/r64020_20190425_132745/1_...,...,false,Sequel II 64020,gs://broad-gp-pacbio/r64020_20190425_132745/1_...,NaN,NaN,NaN,NaN,NaN,NaN,036a73ba-d612-4e3b-834d-738972ca1b30
2,hifiReads,true,71371426975,SM-V35Y,C01,unknown,gs://broad-gp-pacbio/r64218e_20210309_202051/3...,SM-V35Y,3957999,NaN,...,true,64218e,NaN,3957999.0,gs://broad-gp-pacbio/r64218e_20210309_202051/3...,244956.0,1648868.0,2064175.0,gs://broad-gp-pacbio/r64218e_20210309_202051/3...,03777c72-3603-4314-9776-c52e60958340
3,unknown,unknown,40170418330,VP_ABC012_4_1,C01,unknown,gs://broad-gp-pacbio/r64020_20190408_170525/3_C01,VP_ABC012_4_1,3539001,gs://broad-gp-pacbio/r64020_20190408_170525/3_...,...,false,Sequel II 64020,gs://broad-gp-pacbio/r64020_20190408_170525/3_...,NaN,NaN,NaN,NaN,NaN,NaN,03f1ba7b-674c-4405-9c4d-9cf9467099ae
4,unknown,true,405793713318,NA21303,A01,unknown,gs://broad-gp-pacbio/r64020_20200821_205414/1_A01,NA21303,31311159,gs://broad-gp-pacbio/r64020_20200821_205414/1_...,...,false,Sequel II 64020,gs://broad-gp-pacbio/r64020_20200821_205414/1_...,NaN,NaN,NaN,NaN,NaN,NaN,05b4068e-ece1-4238-9cd2-3e6c50fe7bb3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,unknown,unknown,210047960808,NA129878Rep2_HiFi,B01,unknown,gs://broad-gp-pacbio/r64020_20190208_212320/2_B01,NA129878Rep2_HiFi,28050591,gs://broad-gp-pacbio/r64020_20190208_212320/2_...,...,false,Sequel II 64020,gs://broad-gp-pacbio/r64020_20190208_212320/2_...,NaN,NaN,NaN,NaN,NaN,NaN,f9875fb8-ae28-48a4-8c37-c0e21dae5653
146,longReads,false,134571989194,CDH105,C01,CDH105_LR1,gs://broad-gp-pacbio/r64020e_20210409_215247/2...,SM-KTTFO,9498392,gs://broad-gp-pacbio/r64020e_20210409_215247/2...,...,false,64020e,gs://broad-gp-pacbio/r64020e_20210409_215247/2...,NaN,NaN,NaN,NaN,NaN,NaN,fb10e984-b7f1-426b-8a88-e4b7e0602811
147,unknown,false,153540406994,UnknownBioSample,D01,SM-K27NJ\nLCSET-17831,gs://broad-gp-pacbio/r64020_20200403_234856/4_D01,HCT116_celp_shWRN1_clone2,8883515,gs://broad-gp-pacbio/r64020_20200403_234856/4_...,...,false,Sequel II 64020,gs://broad-gp-pacbio/r64020_20200403_234856/4_...,NaN,NaN,NaN,NaN,NaN,NaN,fc154b92-d50e-46de-8745-b5ca6820e824
148,unknown,true,284164220339,SM-HLD5O,A01,SM-HLD5O cell 1 of 2,gs://broad-gp-pacbio/r64020_20190916_181141/1_A01,SM-HLD5O,281848443,gs://broad-gp-pacbio/r64020_20190916_181141/1_...,...,false,Sequel II 64020,gs://broad-gp-pacbio/r64020_20190916_181141/1_...,NaN,NaN,NaN,NaN,NaN,NaN,fd76cad3-940b-43d8-a48a-cddfab87c445


## Examine PacBio subreadset.xml files

Navigate GCS directories and look for PacBio flowcells (indicated by the presence of the *.subreadset.xml and *.consensusreadset.xml files).

In [8]:
ts = load_xmls(gcs_buckets_pb)

In [9]:
tbl_header = ["entity:sample_id", "instrument", "movie_name", "well_name", "created_at", "bio_sample", "well_sample", "insert_size", "is_ccs", "is_isoseq", "is_corrected", "description", "application", "num_records", "total_length", "zmws_input", "zmws_pass", "zmws_fail", "zmws_shortcut_filters", "gcs_input_dir", "subreads_bam", "subreads_pbi", "ccs_bam", "ccs_pbi"]
tbl_rows = []

for e in ts:
    r = load_ccs_report(e['Files']['ccs_reports.txt'])
    
    tbl_rows.append([
        e['CollectionMetadata'][0]['UniqueId'] if 'Context' in e['CollectionMetadata'][0] else "",

        e['CollectionMetadata'][0]['InstrumentName'] if 'Context' in e['CollectionMetadata'][0] else "UnknownInstrument",
        e['CollectionMetadata'][0]['Context'] if 'Context' in e['CollectionMetadata'][0] else "UnknownFlowcell",
        e['WellSample'][0]['WellName'] if 'WellName' in e['WellSample'][0] else "Z00",
        e['WellSample'][0]['CreatedAt'] if 'CreatedAt' in e['WellSample'][0] else "0001-01-01T00:00:00",
        re.sub("[# ]", "", e['BioSample'][0]['Name']) if 'BioSample' in e else "UnknownBioSample",
        re.sub("[# ]", "", e['WellSample'][0]['Name']) if 'Name' in e['WellSample'][0] else "UnknownWellSample",
        e['WellSample'][0]['InsertSize'] if 'InsertSize' in e['WellSample'][0] else "0",
        e['WellSample'][0]['IsCCS'] if 'IsCCS' in e['WellSample'][0] else "unknown",
        e['WellSample'][0]['IsoSeq'] if 'IsoSeq' in e['WellSample'][0] else "unknown",

        "true" if 'ConsensusReadSet' in e else "false",
        e['WellSample'][0]['Description'] if 'Description' in e['WellSample'][0] else "unknown",
        e['WellSample'][0]['Application'] if 'Application' in e['WellSample'][0] else "unknown",
        
        e['DataSetMetadata'][0]['NumRecords'],
        e['DataSetMetadata'][0]['TotalLength'],
        
        r['ZMWs input'],
        r['ZMWs pass filters'],
        r['ZMWs fail filters'],
        r['ZMWs shortcut filters'],

        e['Files']['input_dir'],
        e['Files']['subreads.bam'],
        e['Files']['subreads.bam.pbi'],
        e['Files']['reads.bam'],
        e['Files']['reads.bam.pbi'],
    ])
    
tbl_new = pd.DataFrame(tbl_rows, columns=tbl_header)

In [10]:
tbl_new

,entity:sample_id,instrument,movie_name,well_name,created_at,bio_sample,well_sample,insert_size,is_ccs,is_isoseq,...,total_length,zmws_input,zmws_pass,zmws_fail,zmws_shortcut_filters,gcs_input_dir,subreads_bam,subreads_pbi,ccs_bam,ccs_pbi
0,ae8073d9-4a0d-435e-9837-4f05cd1e8086,Sequel II 64020,m64020_190816_170744,SamplePlate-1-A-1,0001-01-01T00:00:00,EvanAndersen,TestSample,0,unknown,unknown,...,999,,,,,gs://broad-gp-pacbio/3370020/r64020_20190816_1...,gs://broad-gp-pacbio/3370020/r64020_20190816_1...,gs://broad-gp-pacbio/3370020/r64020_20190816_1...,,
1,e254bce4-d677-470b-a5ef-67fe7a57c08c,Sequel II 64020,m64020_190816_173252,SamplePlate-1-A-1,0001-01-01T00:00:00,EvanAndersen,TestSample,0,unknown,unknown,...,443,,,,,gs://broad-gp-pacbio/3370020/r64020_20190816_1...,gs://broad-gp-pacbio/3370020/r64020_20190816_1...,gs://broad-gp-pacbio/3370020/r64020_20190816_1...,,
2,d49e8a58-42c8-487f-bda2-b63a4a4c6085,Sequel II 64020,m64020_200304_203414,SamplePlate-1-A-1,0001-01-01T00:00:00,UnknownBioSample,TestSample,0,unknown,unknown,...,1494,,,,,gs://broad-gp-pacbio/3370020/r64020_20200304_2...,gs://broad-gp-pacbio/3370020/r64020_20200304_2...,gs://broad-gp-pacbio/3370020/r64020_20200304_2...,,
3,cd8b40b5-8519-44cc-99bb-840adc06d5ee,Sequel II 64020,m64020_200304_205638,SamplePlate-1-A-1,0001-01-01T00:00:00,UnknownBioSample,TestSample,0,unknown,unknown,...,0,,,,,gs://broad-gp-pacbio/3370020/r64020_20200304_2...,gs://broad-gp-pacbio/3370020/r64020_20200304_2...,gs://broad-gp-pacbio/3370020/r64020_20200304_2...,,
4,64409e8f-87b1-4913-8114-720672c42c8e,Sequel II 64020,m64020_190123_195048,A01,2019-01-23T19:13:28.717Z,Ecoli_PB_Control,Ecoli_PB_Control,90000,unknown,false,...,145424497417,,,,,gs://broad-gp-pacbio/r64020_20190123_194017/1_A01,gs://broad-gp-pacbio/r64020_20190123_194017/1_...,gs://broad-gp-pacbio/r64020_20190123_194017/1_...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,7e92abbe-d667-40b6-9c70-3113f6ab24a2,64218e,m64218e_210410_202635,B01,2021-04-08T23:56:33.865Z,15349_CCPM_1700406,SM-KUEJZ,10000,false,false,...,60001205573,4262106.0,165127.0,193740.0,3903239.0,gs://broad-gp-pacbio/r64218e_20210409_001100/4...,,,gs://broad-gp-pacbio/r64218e_20210409_001100/4...,gs://broad-gp-pacbio/r64218e_20210409_001100/4...
147,67507ea4-d23b-4a44-88cf-fb7eac33e00f,64218e,m64218e_210413_163605,A01,2021-04-13T13:19:53.907Z,CDH222,SM-KTTGO,20000,false,false,...,146907431260,,,,,gs://broad-gp-pacbio/r64218e_20210413_162418/1...,gs://broad-gp-pacbio/r64218e_20210413_162418/1...,gs://broad-gp-pacbio/r64218e_20210413_162418/1...,,
148,4daefe8c-f187-4cda-aabb-f2872a8d3af0,64218e,m64218e_210414_074430,B01,2021-04-13T13:19:53.907Z,CDH223,SM-KTTGP,20000,false,false,...,108730461974,,,,,gs://broad-gp-pacbio/r64218e_20210413_162418/2...,gs://broad-gp-pacbio/r64218e_20210413_162418/2...,gs://broad-gp-pacbio/r64218e_20210413_162418/2...,,
149,caa0b71a-b18a-4584-b1fc-874511083403,64218e,m64218e_210414_230550,C01,2021-04-13T13:19:53.907Z,47_Father,SM-KTTFQ,20000,false,false,...,59815127600,,,,,gs://broad-gp-pacbio/r64218e_20210413_162418/3...,gs://broad-gp-pacbio/r64218e_20210413_162418/3...,gs://broad-gp-pacbio/r64218e_20210413_162418/3...,,


## Merge old and new sample list

If there are changes to the old sample list, make sure we retain them through subsequent table updates.  Do not overwrite old sample entries (metadata may have been manually modified).

In [11]:
outer_tbl = pd.merge(tbl_old, tbl_new, how='outer', sort=True, indicator=True)

hs = []
for l in list(outer_tbl['entity:sample_id'].unique()):
    g = outer_tbl.loc[outer_tbl['entity:sample_id'] == l]
    
    if len(g) == 1:
        hs.append(g.iloc[0].to_dict())
    else:
        h = {}
        for col_name in list(outer_tbl.columns):
            side = "left_only" if col_name in list(tbl_old.columns) else "right_only"
            q = list(g.loc[g['_merge'] == side][col_name])
            h[col_name] = q[0]

        hs.append(h)
                      
joined_tbl = pd.DataFrame(hs)
del joined_tbl['_merge']

c = list(joined_tbl.columns)
c.remove("entity:sample_id")
c = ["entity:sample_id"] + c
joined_tbl = joined_tbl[c]

## Upload new sample and sample_set tables to Terra

Upload the merged 'sample' table to this workspace.

In [12]:
upload_data(namespace, workspace, joined_tbl)

Uploaded 150 rows successfully.
Uploaded 98 sample sets successfully.
Uploaded 150 sample set members successfully.
